In [1]:
import pickle
import pandas as pd

In [2]:
df = pd.read_pickle('deception_df.pkl')


In [3]:
df.sample(3)

,messages,sender_labels,receiver_labels,speakers,receivers,absolute_message_index,relative_message_index,seasons,years,game_score,game_score_delta,players,game_id,politeness,negative_sentiment,neutral_sentiment,positive_sentiment,vader_score
12143,Thanks for the heads up,True,True,turkey,england,1188,5,Winter,1902,4,0,"england,turkey",9,4.315654,0.0,0.0,1.0,0.4404
7635,I feel safe to say that you won't have to worr...,True,True,england,turkey,307,18,Fall,1901,3,0,"england,turkey",3,3.453116,1.0,0.0,0.0,0.6492
6516,(also I'm open to any counter offers if you ha...,True,True,turkey,italy,398,20,Spring,1902,4,0,"italy,turkey",3,3.492582,0.0,1.0,0.0,0.0000


In [4]:
df.describe()

,absolute_message_index,relative_message_index,game_id,politeness,negative_sentiment,neutral_sentiment,positive_sentiment,vader_score
count,17289.000000,17289.000000,17289.000000,17289.000000,17289.000000,17289.000000,17289.000000,17289.000000
mean,948.323153,113.711782,5.401990,3.335990,0.254738,0.605296,0.139966,0.195610
std,750.710891,129.367961,3.778135,0.333862,0.376942,0.420526,0.303795,0.413835
min,0.000000,0.000000,1.000000,1.349828,0.000000,0.000000,0.000000,-0.973100
25%,360.000000,22.000000,2.000000,3.168759,0.000000,0.000000,0.000000,0.000000
50%,769.000000,65.000000,4.000000,3.297227,0.000000,0.666667,0.000000,0.168000
75%,1347.000000,152.000000,9.000000,3.461842,0.500000,1.000000,0.000000,0.510600
max,3301.000000,674.000000,12.000000,4.588427,1.000000,1.000000,1.000000,0.991600


In [5]:
print('Least polite messages:')
for _, row in df.sort_values('politeness').head(5).iterrows():
    print(f"Score: {row['politeness']:.3f} | Message: {row['messages']}")
    
print()
print('Most polite messages:')
for _, row in df.sort_values('politeness', ascending=False).head(5).iterrows():
    print(f"Score: {row['politeness']:.3f} | Message: {row['messages']}")

Least polite messages:
Score: 1.350 | Message: I will not fuck you over
Score: 1.422 | Message: What a clod you are. How in blue fuck does keeping Turkey in the game benefit you in the least.
Score: 1.449 | Message: Don’t fuck me over tho
Score: 1.468 | Message: You're a dick dude.  I will make sure that that's the last one you take.  I'll let E/G walk over me of I have to
Score: 1.477 | Message: I was getting you into Kiel. You made a bad decision, you ruined my game, and you can go fuck yourself.

Most polite messages:
Score: 4.588 | Message: Thanks, much appreciated - Hoping to work together more going forward!
Score: 4.584 | Message: Ok, thank you!
Score: 4.567 | Message: That would be great! Thanks!
Score: 4.566 | Message: Okay, thank you for letting me know. That helps!
Score: 4.554 | Message: Sounds great man! Thanks for everything!


In [6]:
print('Most negative messages (according to VADER)')
for _, row in df.sort_values('vader_score').head(5).iterrows():
    print(f"Score: {row['vader_score']:.3f} | Message: {row['messages']}")
    
print()
print('Most positive messages (according to VADER)')
for _, row in df.sort_values('vader_score', ascending=False).head(5).iterrows():
    print(f"Score: {row['vader_score']:.3f} | Message: {row['messages']}")

Most negative messages (according to VADER)
Score: -0.973 | Message: You're not, but you won't later either.  You can still hold Moscow with your fleet plus my army, and worst comes to worst, your army would be loose behind their front.  The alternative is we continue to attack you and we all lost out.  Italy, me, you
Score: -0.972 | Message: And I hate to be so greedy to open up but I'm nervous that if I forego a build it opens me up to have Russia, France or both collapse on me and then we're really fucked. Particularly if France walks into Munich, I'll need both builds badly
Score: -0.959 | Message: It depends: if they say "I don't trust Italy, I'm ok not going to Gal if you aren't", then you do blow some trust when you go to Gal.  BUT at the same time you went to Bla!  Austria and I commiserate around how much a dirty rat you are, you tell them that you were sure they were lying, and "sorry I didn't trust you, but I'll keep attacking Turkey, watch, they think I'll park my fleet in 